## CrewAI

Cutting-edge framework for orchestrating role-playing, autonomous AI agents. By fostering collaborative intelligence, CrewAI empowers agents to work together seamlessly, tackling complex tasks.

- Role-Based Agent Design: Customize agents with specific roles, goals, and tools.
- Autonomous Inter-Agent Delegation: Agents can autonomously delegate tasks and inquire amongst themselves, enhancing problem-solving efficiency.
- Flexible Task Management: Define tasks with customizable tools and assign them to agents dynamically.
- Processes Driven: Currently only supports sequential task execution and hierarchical processes, but more complex processes like consensual and autonomous are being worked on.
- Save output as file: Save the output of individual tasks as a file, so you can use it later.
- Parse output as Pydantic or Json: Parse the output of individual tasks as a Pydantic model or as a Json if you want to.
- Works with Open Source Models: Run your crew using Open AI or open source models refer to the Connect crewAI to LLMs page for details on configuring your agents connections to models, even ones running locally!

#### Customer Support Agent System

In this notebook, we will build a two agent system that helps in answering the support queries of agent.

- A junior support agent helps in finding the right answer
- A senior support agent helps in reviewing and answering the question based on junior agent response

In [2]:
import os
from crewai import Agent, Task, Crew
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
os.environ["OPENAI_MODEL_NAME"] = "gpt-3.5-turbo"

In [4]:
junior_support_agent = Agent(
    role="Support Representative",
	goal="Be the most friendly and helpful support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and are now working on providing "
		"support to {customer}, a super important customer for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, and make no assumptions."
	),
	allow_delegation=False,
	verbose=True
)

In [5]:
senior_support_agent = Agent(
	role="Senior Support Specialist",
	goal="Get recognition for providing the best support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and are now working with your team "
		"on a request from {customer} ensuring that the support representative is providing the best support possible.\n"
		"You need to make sure that the support representative is providing full complete answers, and make no assumptions."
	),
	verbose=True,
    allow_delegation=False,
)

In [6]:
from crewai_tools import ScrapeWebsiteTool

docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Coding-Agents/"
)

In [7]:
inquiry_resolution_task = Task(
    description=(
        "{customer} just reached out with a super important ask:\n {inquiry}\n\n"
		"Make sure to use everything you know to provide the best support possible."
		"You must strive to provide a complete and accurate response to the customer's inquiry using the content."
    ),
    expected_output=(
	    "A detailed, informative response to the customer's inquiry that addresses all aspects of their question.\n"
        "The response should include references to everything you used to find the answer, including external data or solutions. "
        "Ensure the answer is complete, leaving no questions unanswered, and maintain a helpful and friendly tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=junior_support_agent,
)

In [8]:
review_task = Task(
    description=(
        "Review the response drafted by the Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to find the information, "
		"ensuring the response is well-supported and leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative answer for the input question ready to be sent to the customer.\n"
        "This response should fully address the customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
    ),
    agent=senior_support_agent,
)


In [9]:
crew = Crew(
  agents=[junior_support_agent, senior_support_agent],
  tasks=[inquiry_resolution_task, review_task],
  verbose=2,
)

In [10]:
inputs = {
    "customer": "SRK",
    "inquiry": "How do I do code execution using crewai. please help."
}
result = crew.kickoff(inputs=inputs)

 [2024-08-11 10:23:14][DEBUG]: == Working Agent: Support Representative
 [2024-08-11 10:23:14][INFO]: == Starting Task: SRK just reached out with a super important ask:
 How do I do code execution using crewai. please help.

Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the customer's inquiry using the content.


> Entering new CrewAgentExecutor chain...
I need to provide SRK with a detailed and informative response about how to do code execution using crewai. I should use the tool available to gather the necessary information.

Action: Read website content
Action Input: {} 

Coding Agents - crewAI
 Skip to content
 crewAI
 Coding Agents
 Initializing search
 crewAI
 crewAI
 crewAI
 Home
 Getting Started
 Getting Started
 Installing CrewAI
 Starting a new CrewAI project
 Core Concepts
 Core Concepts
 Agents
 Tasks
 Tools
 Processes
 Crews
 Collaboration
 Training
 Memory
 Planning
 Testing
 Using

In [11]:
print(result)

Dear SRK,

Thank you for reaching out to crewAI Support with your inquiry. I have reviewed the response drafted by our Support Representative and ensured that it meets the high-quality standards expected for customer support.

Below is a comprehensive and accurate answer to your inquiry regarding enabling code execution for crewAI Agents:

To enable code execution for an agent, you need to set the allow_code_execution parameter to True when creating the agent. It is recommended to use more capable models like Claude 3.5 Sonnet and GPT-4 for code execution tasks as they have a better understanding of programming concepts. The code execution feature includes error handling, allowing agents to receive error messages if the executed code raises an exception. Additionally, installing the crewai_tools package is necessary to use the code execution feature.

The code execution process involves the agent analyzing the task, formulating Python code to solve the problem, sending the code to the 